In [1]:
from copy import deepcopy
import os
from unicodedata import decimal

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

# add imports for randomness
import time
import random

import sys

# Brevitas imports
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from brevitas.quant import Int32Bias
import torch.nn.functional as F

# For adaptive learning rate import
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import random_split


## Imports from utils file for my defined noise functions
import sys
sys.path.append('C:/Users/ashin/source/repos/Cifar10_Pytorch_NoiseAnalysis/Cifar10_Pytorch_NoiseAnalysis/pynq-finn-FPGA/noise_weight_analysis/utils/')

from noise_functions import random_clust_mask, add_mask_to_model_brevitas, mask_noise_plots_brevitas, add_digital_noise, add_digital_noise_to_model_brevitas, ber_noise_plot_brevitas, add_gaussian_noise, add_gaussian_noise_to_model_brevitas, gaussian_noise_plots_brevitas, test 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

Target device: cuda


In [3]:
from torchvision import transforms

# Define data augmentation transforms
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply data augmentation to the training dataset
train_set = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=train_transform)

# Use the validation transform for the validation dataset
val_set =torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=val_transform)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=False, num_workers=4)


a = next(iter(train_loader))
print(a[0].size())
print(len(train_set))

print("Samples in each set: train = %d, test = %s" % (len(train_set), len(train_loader))) 
print("Shape of one input sample: " +  str(train_set[0][0].shape))

## Data Loader
#
# Using PyTorch dataloader we can create a convenient iterator over the dataset that returns batches of data, rather than requiring manual batch creation.

# set batch size
batch_size = 1000

# Create a DataLoader for a training dataset with a batch size of 1000
train_quantized_loader = DataLoader(train_set, batch_size=batch_size)
test_quantized_loader = DataLoader(val_set, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([128, 3, 32, 32])
50000
Samples in each set: train = 50000, test = 391
Shape of one input sample: torch.Size([3, 32, 32])


In [4]:
count = 0

print("\nDataset Shape:\n-------------------------")
for x, y in train_loader:
    print("Input shape for 1 batch: " + str(x.shape))
    print("Label shape for 1 batch: " + str(y.shape))
    count += 1
    if count == 1:
        break



Dataset Shape:
-------------------------
Input shape for 1 batch: torch.Size([128, 3, 32, 32])
Label shape for 1 batch: torch.Size([128])


In [5]:
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        self.quant_inp = qnn.QuantIdentity(bit_width=4, return_quant_tensor=True)

        self.layer1 = qnn.QuantConv2d(3, 32, 3, padding=1, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu1 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.layer2 = qnn.QuantConv2d(32, 32, 3, padding=1, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu2 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.layer3 = qnn.QuantConv2d(32, 64, 3, padding=1, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu3 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.layer4 = qnn.QuantConv2d(64, 64, 3, padding=1, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu4 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.layer5 = qnn.QuantConv2d(64, 64, 3, padding=1, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu5 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.fc1 = qnn.QuantLinear(64 * 8 * 8, 512, bias=True, weight_bit_width=4, bias_quant=Int32Bias)
        self.relu6 = qnn.QuantReLU(bit_width=4, return_quant_tensor=True)

        self.fc2 = qnn.QuantLinear(512, 10, bias=True, weight_bit_width=4, bias_quant=Int32Bias)

    def forward(self, x):
        x = self.quant_inp(x)
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = F.max_pool2d(x, 2)

        x = self.relu3(self.layer3(x))
        x = self.relu4(self.layer4(x))
        x = F.max_pool2d(x, 2)

        x = self.relu5(self.layer5(x))

        x = x.view(x.size(0), -1)

        x = self.relu6(self.fc1(x))
        x = self.fc2(x)

        return x

In [6]:
# Import testing
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import precision_recall_fscore_support

# Initialize the model, optimizer, and criterion
model = CIFAR10CNN().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)
criterion = nn.CrossEntropyLoss()

num_epochs = 80
best_test_accuracy = 0
patience = 8
no_improvement_counter = 0

print(model)

CIFAR10CNN(
  (quant_inp): QuantIdentity(
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (act_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (restrict_value_impl): FloatRestrictValue()
            )
            (clamp_scaling): _ClampValue(
              (clamp_min_

In [7]:
for epoch in range(num_epochs):

    # training phase
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
    
    # Initialize the validation loss
    val_loss = 0
    
    # testing phase
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()  # accumulate the validation loss
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    
        val_loss /= len(val_loader)  # calculate the average validation loss
    
        # Update the learning rate using the validation loss
        scheduler.step(val_loss)

        test_accuracy = 100 * correct / total
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
        
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1
            
        if no_improvement_counter >= patience:
            print("Early stopping")
            break

        print('Epoch [{}/{}], Test Accuracy: {:.2f}%, Precision: {:.2f}, Recall: {:.2f}, F1 score: {:.2f}'.format(epoch+1, num_epochs, test_accuracy, precision, recall, f1))


Epoch [1/80], Step [100/391], Loss: 1.6589
Epoch [1/80], Step [200/391], Loss: 1.6362
Epoch [1/80], Step [300/391], Loss: 1.4192
Epoch [1/80], Test Accuracy: 49.77%, Precision: 0.50, Recall: 0.50, F1 score: 0.49
Epoch [2/80], Step [100/391], Loss: 1.3538
Epoch [2/80], Step [200/391], Loss: 1.3180
Epoch [2/80], Step [300/391], Loss: 1.3583
Epoch [2/80], Test Accuracy: 58.86%, Precision: 0.60, Recall: 0.59, F1 score: 0.58
Epoch [3/80], Step [100/391], Loss: 1.1688
Epoch [3/80], Step [200/391], Loss: 1.0445
Epoch [3/80], Step [300/391], Loss: 1.0118
Epoch [3/80], Test Accuracy: 63.20%, Precision: 0.64, Recall: 0.63, F1 score: 0.63
Epoch [4/80], Step [100/391], Loss: 1.0541
Epoch [4/80], Step [200/391], Loss: 0.7828
Epoch [4/80], Step [300/391], Loss: 0.9463
Epoch [4/80], Test Accuracy: 67.58%, Precision: 0.68, Recall: 0.68, F1 score: 0.67
Epoch [5/80], Step [100/391], Loss: 0.8469
Epoch [5/80], Step [200/391], Loss: 1.0458
Epoch [5/80], Step [300/391], Loss: 0.9381
Epoch [5/80], Test Accu

Epoch [39/80], Step [100/391], Loss: 0.3271
Epoch [39/80], Step [200/391], Loss: 0.3271
Epoch [39/80], Step [300/391], Loss: 0.2167
Epoch [39/80], Test Accuracy: 84.91%, Precision: 0.85, Recall: 0.85, F1 score: 0.85
Epoch [40/80], Step [100/391], Loss: 0.2513
Epoch [40/80], Step [200/391], Loss: 0.2719
Epoch [40/80], Step [300/391], Loss: 0.2433
Epoch [40/80], Test Accuracy: 84.89%, Precision: 0.85, Recall: 0.85, F1 score: 0.85
Epoch [41/80], Step [100/391], Loss: 0.3585
Epoch [41/80], Step [200/391], Loss: 0.3375
Epoch [41/80], Step [300/391], Loss: 0.3184
Epoch [41/80], Test Accuracy: 85.27%, Precision: 0.85, Recall: 0.85, F1 score: 0.85
Epoch [42/80], Step [100/391], Loss: 0.3382
Epoch [42/80], Step [200/391], Loss: 0.3540
Epoch [42/80], Step [300/391], Loss: 0.5220
Epoch [42/80], Test Accuracy: 85.03%, Precision: 0.85, Recall: 0.85, F1 score: 0.85
Epoch [43/80], Step [100/391], Loss: 0.3500
Epoch [43/80], Step [200/391], Loss: 0.2108
Epoch [43/80], Step [300/391], Loss: 0.4878
Epoc

In [8]:
# Print the best test accuracy
print("The final best test accuracy is: {:.2f}%".format(best_test_accuracy))

The final best test accuracy is: 85.87%


In [9]:
trained_state_dict = model.state_dict()

## Saving Model

In [10]:
if test_accuracy > best_test_accuracy:
    best_test_accuracy = test_accuracy
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_test_accuracy': best_test_accuracy,
        'epoch': epoch
        }, 'best_model.pth')
    no_improvement_counter = 0


In [12]:
# Create an instance of your neural network model
model = CIFAR10CNN().to(device)

# Load the saved state dictionary from file
state_dict = torch.load('best_model.pth')

# Load the state dictionary into the model
model.load_state_dict(state_dict)


<All keys matched successfully>

# Plots

## Mask Code Here for Editing

In [26]:
def random_clust_mask(weight_shape, P, gamma):

    # Generate random NxN matrix with values between 0 and 1
    N = weight_shape[-1]
    matrix = np.random.rand(N, N)

    # Compute 2D FFT
    fft_result = np.fft.fft2(matrix)

    # 1D Frequency Vector with N bins
    f = np.fft.fftfreq(N, d=1.0/weight_shape[-1])
    f_x, f_y = np.meshgrid(f, f)
    f_x[0, 0] = 1e-6
    f_y[0, 0] = 1e-6

    # Create a 2D filter in frequency space that varies inversely with freq over f
    # Gamma controls the falloff rate
    filter_2D = 1/(np.sqrt(f_x**2 + f_y**2))**gamma

    # Mult the 2D elementwise by the filter
    filtered_fft = fft_result * filter_2D

    # 2D inverse FFT of the filtered result
    ifft_result = np.fft.ifft2(filtered_fft)
    ifft_result = np.real(ifft_result)

    # Set the threshold T equal the the max value in IFFT
    T = ifft_result.max()

    # Init empty bool mask with same dims as ifft
    mask = np.zeros_like(ifft_result, dtype=bool)

    decrement_step = 0.01

    # Repeat until frac of nonzero values in the mask is greater than or equal to P
    while True:
        mask = ifft_result > T

        current_fraction = np.count_nonzero(mask) / (N * N)

        if current_fraction >= P:
            break

        T -= decrement_step

    # Return tensor with the same shape as the input tensor
    mask = np.tile(mask, (weight_shape[0], weight_shape[1], 1, 1))
    return torch.from_numpy(mask).float().clone().detach()



"""
This function takes a brevitas layer, a mask generated by the random_clust_mask function,
and two parameters P and gamma. It generates a random clustered mask using the random_clust_mask
function, and then sets the weights at the locations in the mask to zero, effectively sparsifying
the layer's weights.
"""

def add_mask_to_model_brevitas(model, layer_names, p, gamma, num_perturbations, print_weights=False):

    modified_models = []

    for _ in range(num_perturbations):

        modified_model = deepcopy(model)

        for layer_name in layer_names:

            layer = getattr(modified_model, layer_name)

            with torch.no_grad():

                # get weights of the tensors
                weight_tensor = layer.weight.clone().detach()

                # generate mask with correct shape
                mask = random_clust_mask(weight_tensor.shape, p, gamma)

                if print_weights:
                    print("Weights before masking:")
                    print(weight_tensor)

                # apply mask to the whole weight tensor
                mask_tensor = torch.tensor(mask, dtype=torch.float).to(device)
                weight_tensor *= mask_tensor

                if print_weights:
                    print("Weights after masking:")
                    print(weight_tensor)

                # create new weight parameter and assign to layer
                noised_weight = torch.nn.Parameter(weight_tensor, requires_grad=False)
                layer.weight = noised_weight

        modified_models.append(modified_model)

    return modified_models

"""
The function mask_noise_plots_brevitas() is a Python function that is used to visualize the effect of different perturbations on the accuracy of a neural network. 
It generates several visualizations, including a heatmap and a scatter plot.

The function takes in the following parameters:

    num_perturbations: an integer indicating the number of perturbations to apply to the neural network
    layer_names: a list of strings indicating the names of the layers in the neural network
    p_vals: a list of floats indicating the p values to use for the mask perturbation
    gamma_vals: a list of floats indicating the gamma values to use for the mask perturbation
    model: the neural network model to test
    device: the device to use for testing the model

The function begins by creating a directory for saving the output plots. 
It then initializes an empty list called all_test_accs to store the test accuracies for each layer.

For each layer in layer_names, the function initializes an empty list called test_accs to store 
the test accuracies for each mask perturbation. The function then iterates over each p and gamma value 
and adds noise to the model for the defined layer only. It then tests the accuracy of each noisy model 
and appends the result to the accuracies list. The function then calculates the average accuracy 
and prints the result.

The test accuracies for the current layer are then stored in the all_test_accs list.

The function then creates a heatmap for each layer and saves it to disk.
It also computes the average test accuracy across all layers for each p and gamma value and creates a
heatmap for the average test accuracy. Both the individual and average heatmaps have a color bar 
indicating the test accuracy.

Finally, the function creates a scatter plot showing the test accuracies for each layer at each p 
and gamma value. The plot has layer_names on the x-axis, p_vals on the y-axis, and gamma_vals on the z-axis. 
The points in the plot are colored based on the corresponding test accuracy, with a color bar indicating 
the mapping between color and test accuracy.
"""

def test(model, test_loader, device):
    # testing phase
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy





def mask_noise_plots_brevitas(num_perturbations, layer_names, p_values, gamma_values, model, device):
    
    
    for p in range(len(p_values)):
        for g in range(len(gamma_values)):
            
            noisy_models = add_mask_to_model_brevitas(model, layer_names, p_values[p], gamma_values[g], num_perturbations)
            
            accuracies = []
            
            for noisy_model in noisy_models:
                
                noisy_model.to(device)

                accuracies.append(test(noisy_model, test_quantized_loader, device))
                
            avg_accuracy = sum(accuracies)/len(accuracies)
            
            print("P Value: {}\t Gamma Value: {}\t Average Accuracy: {}%".format(p_values[p], gamma_values[g], avg_accuracy))

In [27]:
## Mask
layer_names = ['layer1', 'layer2', 'layer3', 'layer4', 'layer5']
perturbations = 30
p_values = np.linspace(1e-3, 0.4, 15)
gamma_values = np.linspace(1e-3, 0.99, 5)


mask_noise_plots_brevitas(1, layer_names, p_values, gamma_values, model, device)

C:\Users\ashin\AppData\Local\Temp\ipykernel_14108\2427530726.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(mask, dtype=torch.float).to(device)


P Value: 0.001	 Gamma Value: 0.001	 Average Accuracy: 10.03%
P Value: 0.001	 Gamma Value: 0.24825	 Average Accuracy: 10.0%
P Value: 0.001	 Gamma Value: 0.4955	 Average Accuracy: 9.86%
P Value: 0.001	 Gamma Value: 0.74275	 Average Accuracy: 10.0%
P Value: 0.001	 Gamma Value: 0.99	 Average Accuracy: 10.0%
P Value: 0.029500000000000002	 Gamma Value: 0.001	 Average Accuracy: 10.0%
P Value: 0.029500000000000002	 Gamma Value: 0.24825	 Average Accuracy: 10.0%
P Value: 0.029500000000000002	 Gamma Value: 0.4955	 Average Accuracy: 10.0%
P Value: 0.029500000000000002	 Gamma Value: 0.74275	 Average Accuracy: 10.0%
P Value: 0.029500000000000002	 Gamma Value: 0.99	 Average Accuracy: 10.0%


KeyboardInterrupt: 

In [29]:
def ber_noise_plot_brevitas(num_perturbations, layer_names, ber_vector, model, device):
    
    if not os.path.exists("noise_plots_brevitas/ber_noise/"):
        os.makedirs("noise_plots_brevitas/ber_noise/")
    
    plt.style.use('default')
    
    all_test_accs = []

    for layer in layer_names:
        test_accs = []
        
        for ber in ber_vector:
            noisy_models = add_digital_noise_to_model_brevitas(model, [layer], ber, num_perturbations)
            
            accuracies = []
            
            for noisy_model in noisy_models:
                
                noisy_model.to(device)
                accuracies.append(test(noisy_model, test_quantized_loader, device))
                
            avg_accuracy = sum(accuracies) / len(accuracies)
            
            test_accs.append(avg_accuracy)
            
            print("BER Value: {}\tAverage Accuracy: {}".format(ber, avg_accuracy))
            
        all_test_accs.append(test_accs)
        
        plt.plot(ber_vector, test_accs,
                 label='{} Accuracy at Different Perturbation Levels'.format(layer))
        
        plt.xlabel('Standard Deviation')
        plt.ylabel('Test Accuracy')
        plt.title('Effect of Noise on Test Accuracy')
        plt.legend()
        plt.savefig("noise_plots_brevitas/ber_noise/{}.png".format(layer))
        plt.clf()
        print('Done with Plot {}'.format(layer))
        
    avg_test_accs = [sum(x) / len(x) for x in zip(*all_test_accs)]
    
    plt.plot(ber_vector, avg_test_accs, label='Average',
             linewidth=3, linestyle='--', color="black")
    
    plt.xlabel('BER Value')
    
    plt.ylabel('Test Accuracy')
    
    plt.title('Effect of BER Noise on Test Accuracy (Average)')
    
    plt.legend()
    plt.savefig("noise_plots_brevitas/ber_noise/average.png")
    plt.clf()

In [30]:
## Digital Noise with BER
layer_names = ['layer1', 'layer2', 'layer3', 'layer4', 'layer5', 'fc1', 'fc2']
ber_vals = [1e-6, 1e-5, 1e-4, 1e-3, 0.01, 0.1]
perturbations = 10

ber_noise_plot_brevitas(perturbations, layer_names, ber_vals, model, device)

BER Value: 1e-06	Average Accuracy: 85.88
BER Value: 1e-05	Average Accuracy: 85.88
BER Value: 0.0001	Average Accuracy: 85.75899999999999
BER Value: 0.001	Average Accuracy: 84.695
BER Value: 0.01	Average Accuracy: 78.97200000000001
BER Value: 0.1	Average Accuracy: 41.394
Done with Plot layer1
BER Value: 1e-06	Average Accuracy: 85.88
BER Value: 1e-05	Average Accuracy: 85.88
BER Value: 0.0001	Average Accuracy: 85.48299999999998
BER Value: 0.001	Average Accuracy: 84.44500000000001
BER Value: 0.01	Average Accuracy: 80.296
BER Value: 0.1	Average Accuracy: 36.361000000000004
Done with Plot layer2
BER Value: 1e-06	Average Accuracy: 85.88
BER Value: 1e-05	Average Accuracy: 85.736
BER Value: 0.0001	Average Accuracy: 85.481
BER Value: 0.001	Average Accuracy: 84.73599999999999
BER Value: 0.01	Average Accuracy: 82.798
BER Value: 0.1	Average Accuracy: 62.157999999999994
Done with Plot layer3
BER Value: 1e-06	Average Accuracy: 85.88
BER Value: 1e-05	Average Accuracy: 85.664
BER Value: 0.0001	Average A

<Figure size 640x480 with 0 Axes>

In [ ]:
## Gaussian Noise
sigma_vector = np.linspace(0, 0.2, 11)